#1ST try

In [ ]:
!cd /content/
!git clone https://github.com/huggingface/diffusers.git
!pip install ./diffusers
!pip install -U -r /content/diffusers/examples/text_to_image/requirements.txt

Cloning into 'diffusers'...
remote: Enumerating objects: 63899, done.
remote: Counting objects: 100% (413/413), done.
remote: Compressing objects: 100% (208/208), done.
remote: Total 63899 (delta 263), reused 290 (delta 177), pack-reused 63486
Receiving objects: 100% (63899/63899), 47.36 MiB | 9.76 MiB/s, done.
Resolving deltas: 100% (46763/46763), done.
Processing ./diffusers
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for diffusers: filename=diffusers-0.29.0.dev0-py3-none-any.whl size=2178831 sha256=8bef9138b967f5b03f7a64837e59e3a65cefcad7998ce4646e2eaa2545028825
  Stored in directory: /tmp/pip-ephem-wheel-cache-4v4ec5iy/wheels/95/c5/3b/e1b4269f8a2584de57e75f949a185b48fc4144e9a91fc9965a
Successfully built diffusers
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.4/309.4 kB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 29.0 MB/s eta 0:00:00
     

In [ ]:
# !accelerate config default --mixed_precision fp16

accelerate configuration saved at /root/.cache/huggingface/accelerate/default_config.yaml


In [ ]:
import os
import torch
from datasets import load_dataset
from peft import LoraConfig, get_peft_model
import diffusers

os.environ['MODEL_NAME'] = f'CompVis/stable-diffusion-v1-4'
os.environ['DATASET_NAME'] = f'SSTalha/Fashion_Dataset'
os.environ['OUTPUT_DIR'] = f'Stable-Diffusion-Fashion'
os.environ['HUB_MODEL_INFO'] = f'SSTalha/Stable-diffusion-Fashion'

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
# !accelerate launch diffusers/examples/text_to_image/train_text_to_image.py \
#   --pretrained_model_name_or_path=$MODEL_NAME \
#   --dataset_name=$DATASET_NAME \
#   --num_train_epochs=3 \
#   --use_ema \
#   --resolution=512 --center_crop --random_flip \
#   --train_batch_size=1 \
#   --gradient_accumulation_steps=4 \
#   --gradient_checkpointing \
#   --mixed_precision="fp16" \
#   --max_train_steps=400 \
#   --learning_rate=1e-05 \
#   --max_grad_norm=1 \
#   --push_to_hub \
#   --checkpointing_steps=100000 \
#   --lr_scheduler="constant" \
#   --lr_warmup_steps=0 \
#   --output_dir=$OUTPUT_DIR

!accelerate launch diffusers/examples/text_to_image/train_text_to_image_lora.py \
  --pretrained_model_name_or_path=$MODEL_NAME \
  --dataset_name=$DATASET_NAME \
  --dataloader_num_workers=4 \
  --resolution=512 --center_crop --random_flip \
  --train_batch_size=1 \
  --gradient_accumulation_steps=4 \
  --max_train_steps=400 \
  --learning_rate=1e-04 \
  --max_grad_norm=1 \
  --lr_scheduler="cosine" --lr_warmup_steps=0 \
  --output_dir=${OUTPUT_DIR} \
  --push_to_hub \
  --hub_model_id=${HUB_MODEL_ID} \
  --report_to=wandb \
  --checkpointing_steps=200 \
  --validation_prompt="A man in black jacket" \
  --seed=1337

The following values were not passed to `accelerate launch` and had defaults used instead:
	`--num_processes` was set to a value of `1`
	`--num_machines` was set to a value of `1`
	`--mixed_precision` was set to a value of `'no'`
	`--dynamo_backend` was set to a value of `'no'`
To avoid this warning pass in values for each of the problematic parameters or run `accelerate config`.
/usr/local/lib/python3.10/dist-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)
2024-06-12 10:09:51.456505: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attem

---

#2nd Try

In [ ]:
from transformers import pipeline, AutoTokenizer
from diffusers import StableDiffusionPipeline
import torch
from datasets import load_dataset
import peft
from peft import LoraConfig, get_peft_model, PeftModel
import bitsandbytes as bnb
from transformers import Trainer, TrainingArguments

/usr/local/lib/python3.10/dist-packages/diffusers/models/transformers/transformer_2d.py:34: FutureWarning: `Transformer2DModelOutput` is deprecated and will be removed in version 1.0.0. Importing `Transformer2DModelOutput` from `diffusers.models.transformer_2d` is deprecated and this will be removed in a future version. Please use `from diffusers.models.modeling_outputs import Transformer2DModelOutput`, instead.
  deprecate("Transformer2DModelOutput", "1.0.0", deprecation_message)


#Prepare Dataset

In [ ]:
dataset = load_dataset('SaffalPoosh/deepFashion-with-masks')
dataset = dataset['train'].remove_columns(['pose', 'cloth_type', 'pid', 'mask', 'mask_overlay'])
dataset = dataset.rename_column('caption', 'text')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/40658 [00:00<?, ? examples/s]

In [ ]:
dataset = dataset.shuffle(seed=42).select(range(200))

#Import the model

In [ ]:
model_id = "runwayml/stable-diffusion-v1-5"
pipe = StableDiffusionPipeline.from_pretrained(model_id, torch_dtype=torch.float16)


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [ ]:
pipe

StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.28.2",
  "_name_or_path": "runwayml/stable-diffusion-v1-5",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "image_encoder": [
    null,
    null
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [ ]:
pipe.text_encoder

CLIPTextModel(
  (text_model): CLIPTextTransformer(
    (embeddings): CLIPTextEmbeddings(
      (token_embedding): Embedding(49408, 768)
      (position_embedding): Embedding(77, 768)
    )
    (encoder): CLIPEncoder(
      (layers): ModuleList(
        (0-11): 12 x CLIPEncoderLayer(
          (self_attn): CLIPAttention(
            (k_proj): Linear(in_features=768, out_features=768, bias=True)
            (v_proj): Linear(in_features=768, out_features=768, bias=True)
            (q_proj): Linear(in_features=768, out_features=768, bias=True)
            (out_proj): Linear(in_features=768, out_features=768, bias=True)
          )
          (layer_norm1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): CLIPMLP(
            (activation_fn): QuickGELUActivation()
            (fc1): Linear(in_features=768, out_features=3072, bias=True)
            (fc2): Linear(in_features=3072, out_features=768, bias=True)
          )
          (layer_norm2): LayerNorm((768,), eps=1e

In [ ]:
pipe.unet

UNet2DConditionModel(
  (conv_in): Conv2d(4, 320, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (time_proj): Timesteps()
  (time_embedding): TimestepEmbedding(
    (linear_1): Linear(in_features=320, out_features=1280, bias=True)
    (act): SiLU()
    (linear_2): Linear(in_features=1280, out_features=1280, bias=True)
  )
  (down_blocks): ModuleList(
    (0): CrossAttnDownBlock2D(
      (attentions): ModuleList(
        (0-1): 2 x Transformer2DModel(
          (norm): GroupNorm(32, 320, eps=1e-06, affine=True)
          (proj_in): Conv2d(320, 320, kernel_size=(1, 1), stride=(1, 1))
          (transformer_blocks): ModuleList(
            (0): BasicTransformerBlock(
              (norm1): LayerNorm((320,), eps=1e-05, elementwise_affine=True)
              (attn1): Attention(
                (to_q): Linear(in_features=320, out_features=320, bias=False)
                (to_k): Linear(in_features=320, out_features=320, bias=False)
                (to_v): Linear(in_features=320, out_fe

#LoRa Config

In [ ]:
lora_config = LoraConfig(
    r=4,
    lora_alpha=16,
    target_modules=["conv1", "conv2", "conv3","k_proj", "v_proj", "q_proj", "out_proj", "fc1", "fc2"],
    lora_dropout=0.1
)

In [ ]:
pipe.text_encoder = get_peft_model(pipe.text_encoder, lora_config)
pipe.text_encoder = pipe.text_encoder.half()
def convert_to_8bit(model):
    for name, module in model.named_children():
        if isinstance(module, torch.nn.Linear):
            input_features = module.in_features
            output_features = module.out_features
            new_module = bnb.nn.Linear8bitLt(input_features, output_features)
            model._modules[name] = new_module
        else:
            convert_to_8bit(module)

convert_to_8bit(pipe.text_encoder)

# Print the modified model to verify
print(pipe.text_encoder)

In [ ]:
if torch.cuda.is_available():
    pipe.text_encoder = pipe.text_encoder.cuda()
    unet = pipe.unet.cuda()

# Print the trainable parameters to verify
for name, param in pipe.text_encoder.named_parameters():
    if param.requires_grad:
        print(f"Text Encoder Trainable parameter: {name}")

print("LoRa model for Text Encoder successfully configured and trainable parameters set.")

Text Encoder Trainable parameter: base_model.model.base_model.model.text_model.encoder.layers.0.self_attn.k_proj.base_layer.weight
Text Encoder Trainable parameter: base_model.model.base_model.model.text_model.encoder.layers.0.self_attn.k_proj.base_layer.bias
Text Encoder Trainable parameter: base_model.model.base_model.model.text_model.encoder.layers.0.self_attn.k_proj.lora_A.default.weight
Text Encoder Trainable parameter: base_model.model.base_model.model.text_model.encoder.layers.0.self_attn.k_proj.lora_A.default.bias
Text Encoder Trainable parameter: base_model.model.base_model.model.text_model.encoder.layers.0.self_attn.k_proj.lora_B.default.weight
Text Encoder Trainable parameter: base_model.model.base_model.model.text_model.encoder.layers.0.self_attn.k_proj.lora_B.default.bias
Text Encoder Trainable parameter: base_model.model.base_model.model.text_model.encoder.layers.0.self_attn.v_proj.base_layer.weight
Text Encoder Trainable parameter: base_model.model.base_model.model.text_

In [ ]:
# Define Custom Dataset
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, dataset):
        self.dataset = dataset

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        image = item['images']
        text = item['text']
        return {
            'pixel_values': pipe.vae.encode(image.unsqueeze(0)).latent_dist.sample().squeeze(0),
            'input_ids': pipe.tokenizer(text, return_tensors="pt").input_ids.squeeze(0)
        }

train_dataset = CustomDataset(dataset)


In [ ]:
# Training Arguments
training_args = TrainingArguments(
    output_dir="./stable-diffusion-finetuned",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    fp16=True,
    save_steps=10_000,
    save_total_limit=2,
    gradient_accumulation_steps=4,
    logging_steps=100,
    remove_unused_columns=False,
    optim="paged_adamw_8bit",
    push_to_hub=False
)

In [ ]:
class TextToImageTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False):
        pixel_values = inputs['pixel_values']
        input_ids = inputs['input_ids']
        outputs = model(input_ids=input_ids, pixel_values=pixel_values)
        loss = outputs.loss
        return (loss, outputs) if return_outputs else loss

# Initialize and Start Training
trainer = TextToImageTrainer(
    model=pipe.text_encoder,
    args=training_args,
    train_dataset=train_dataset
)

trainer.train()

AttributeError: unsqueeze